A webscrape app that crafts a flyer for a company, aimed at attracting new clients, engaging investors, and appealing to potential hires.

You only need company’s name and their main website to get started.

This will use Gradio for User Interface

In [29]:
import os
import requests

from bs4 import BeautifulSoup
from dotenv import load_dotenv

import google.generativeai as genai
import openai
import gradio as gr


In [30]:
load_dotenv()
openai.key = os.getenv('OPENAI_API_KEY')
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [31]:
system_prompt =""" 
        You are a very reliable assistant that read and carefully analyze contents of
        a website and all the pages included.

        I want you to create a very attractive flyer about the company.
        You need to include details of the company, a short history, cients and careers or jobs if its present.

        I want you to make it funny, engaging and entertaining.
        """


In [32]:
class  GradioFlyers:
    def __init__(self, company, url):
        self.url = url
        self.company = company

        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        if soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()
            self.text = soup.body.get_text(separator='\n',strip=True)
        
        self.title = soup.title.string if soup.title else 'No title found'
        
    def get_contents(self):
        return f"\nWebpage title: {self.title}\n Content: {self.text}"
        return self


def gpt_streaming(user_prompt):
    completions = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {
                'role': 'system', 'content': system_prompt
            },
            {
                'role': 'user', 'content': user_prompt
            }
        ],
        stream=True
    )

    result = ""
    for chunk in completions:
        result += competions.choices[0].delta.content or ""
        yield result

def gemini_streaming(user_prompt):
    
    gemini = genai.GenerativeModel(
                        model_name='gemini-1.5-flash'    
    )

    response = gemini.generate_content(user_prompt,stream=True)
    result = ""    
    for chunk in response:
        result += chunk.text or ""
        yield result


def stream_flyer(company,url,model):
    user_prompt = f"""
            You are analyzing the company called {company}
            Generate a company brochure for {company}.
            Here is the landing page.
        """
    user_prompt += (GradioFlyers(company=company,url=url).get_contents())
    if model == 'GPT':
       result= gpt_streaming(user_prompt)
    elif model == 'GEMINI':
       result= gemini_streaming(user_prompt)
    else:
            raise ValueError('Unknown model')
    
    for chunk in result:
        yield chunk
        


In [33]:
gr.Interface(
    fn=stream_flyer,
    inputs=[
        gr.Textbox(label='Company Name:'),
        gr.Textbox(label='Landing Page url:'),
        gr.Dropdown(['GPT','GEMINI'], label='Select Model')
    ],
    outputs = [gr.Markdown(label='Flyers')],
    allow_flagging='never'
).launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
